## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## 1 Base de dados com dados limpos.

In [ ]:
# Base Suja
carros = pd.read_csv('https://raw.githubusercontent.com/RafaelSilvax06/ProjetoIA-Analise/refs/heads/main/car_sales_data.csv')

#Aplicando a mesma limpeza da fase 1

# Tradução de colunas
carros.columns = carros.columns = ['Fabricante', 'Modelo', 'Potencia Motor', 'Tipo Combustivel', 'Ano De Fabricacao', 'Quilometragem', 'Preco']

# Convertando preço em reais.
carros['Preco'] = carros['Preco'].astype(float)
carros['Preco'] = carros['Preco'] * 5.38

# Converter Quilometragem para float.
carros['Quilometragem'] = carros['Quilometragem'].astype(float)

# Removendo valores irreais (Preços abaixo de R$ 3.500,00)
carros_valor_irreais = carros[carros['Preco'] <= 3500.00].sort_values(by='Preco')
carros.drop(index=carros_valor_irreais.index, inplace=True)


print('Dados limpos e prontos para o pre processamento: ')
print('\n')
carros.info()

## 2 Definição de X e Y
Separando o conjunto de dados.


In [ ]:
#Definindo X e y para criar o modelo e trabalhar na regressão.

# Y é o preço, nosso alvo.
y = carros['Preco']
# X é nossas pistas, colunas correlacionadas ao preço.
X = carros.drop('Preco', axis=1)


### 2.1 Divisão de treino com sklearn

In [ ]:
#Usamos 80% para treino e 20% para teste.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Tamanho de X_train (treino): {X_train.shape[0]} amostras") # 80% das amostras da base de dados.
print(f"Tamanho de X_test (teste): {X_test.shape[0]} amostras") # 20% das amostras da base de dados.
